In [ ]:
"""
This script fills out the hypothetical_regular_form field in cards.csv
Each cell is for a different tense/person combination.
"""

In [ ]:
"""
Add gerundio forms to the cards.csv file, including reflexive verbs (ending in 'se').
"""

import pandas as pd

# Load the CSV
df = pd.read_csv('cards.csv')

def get_gerundio(verb):
    # Handle reflexive verbs ending in 'se'
    is_reflexive = False
    if verb.endswith('se'):
        is_reflexive = True
        base = verb[:-2]  # Remove 'se'
    else:
        base = verb
    if base.endswith('ar'):
        form = base[:-2] + 'ando'
    elif base.endswith('er') or base.endswith('ir') or base.endswith('ír'):
        form = base[:-2] + 'iendo'
    else:
        form = ''
    if is_reflexive:
        form += 'se'
    return form

# Only update rows where tense is 'gerundio'
mask = df['tense'] == 'gerundio'
df.loc[mask, 'hypothetical_regular_form'] = df.loc[mask, 'verb'].apply(get_gerundio)

# Save the updated CSV
df.to_csv('cards.csv', index=False)

In [3]:
def get_gerundio(verb):
    # Handle reflexive verbs ending in 'se'
    is_reflexive = False
    if verb.endswith('se'):
        is_reflexive = True
        base = verb[:-2]  # Remove 'se'
    else:
        base = verb
    if base.endswith('ar'):
        form = base[:-2] + 'ando'
    elif base.endswith('er') or base.endswith('ir') or base.endswith('ír'):
        form = base[:-2] + 'iendo'
    else:
        form = ''
    if is_reflexive:
        form += 'se'
    return form

verb = 'irse'
print(get_gerundio(verb))  # Output: hablando

iendose
